<a href="https://colab.research.google.com/github/iqbalamo93/keras_RNN/blob/master/NLP_RNN_text_gen_kr_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf

In [3]:
#Download any text from guttenberg, we choose Shakespear

In [4]:
path_to_file = 'https://datasets007.s3.amazonaws.com/shakespeare.txt'

In [5]:
from urllib.request import urlopen

text = urlopen(path_to_file).read().decode('utf-8')


In [6]:
print(text[40500:41000])

umbers should be broken,
  While shadows like to thee do mock my sight?
  Is it thy spirit that thou send'st from thee
  So far from home into my deeds to pry,
  To find out shames and idle hours in me,
  The scope and tenure of thy jealousy?
  O no, thy love though much, is not so great,
  It is my love that keeps mine eye awake,
  Mine own true love that doth my rest defeat,
  To play the watchman ever for thy sake.
    For thee watch I, whilst thou dost wake elsewhere,
    From me far off, wi


In [7]:
vocab = sorted(set(text))

In [8]:
vocab

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}']

In [9]:
len(vocab)

84

In [10]:
char_to_index = {char:id for id,char in enumerate(vocab)}

In [11]:
char_to_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [12]:
ind_to_char = np.array(vocab)

In [13]:
ind_to_char[33]

'H'

In [14]:
encoded_text = np.array([char_to_index[x] for x in text])

In [15]:
encoded_text.shape

(5445609,)

In [16]:
encoded_text[:500]

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64, 62, 63,
       75,  1, 69, 60, 77, 60, 73,  1, 59, 64, 60,  8,  0,  1,  1, 27, 76,
       75,  1, 56, 74,  1, 75, 63, 60,  1, 73, 64, 71, 60, 73,  1, 74, 63,
       70, 76, 67, 59,  1, 57, 80,  1, 75, 64, 68, 60,  1, 59, 60, 58, 60,
       56, 74, 60,  8,  0,  1,  1, 33, 64, 74,  1, 75, 60, 69, 59, 60, 73,
        1, 63, 60, 64, 73,  1, 68, 64, 62, 63, 75,  1, 57, 60, 56, 73,  1,
       63, 64, 74,  1, 68, 60, 68, 70, 73, 80, 21,  0,  1,  1, 27, 76, 75,
        1, 75, 63, 70, 76,  1, 58, 70, 69, 75, 73, 56, 58, 75, 60, 59,  1,
       75, 70,  1, 75, 63

In [17]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [18]:
seq_len = 120 #sequence length

In [19]:
total_num_seq = len(text)//(seq_len+1)

In [20]:
total_num_seq

45005

In [21]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [22]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [ ]:
for item in  char_dataset.take(500):#take create dataset with  almost element 
  print(ind_to_char[item.numpy()])

In [24]:
sequences = char_dataset.batch(seq_len+1,drop_remainder=True)

In [25]:
def create_seq_target(seq):
  input_txt = seq[:-1]
  target_txt = seq[1:]
  return input_txt,target_txt

In [26]:
type(sequences)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [27]:
dataset =  sequences.map(create_seq_target)

In [28]:
dataset

<MapDataset shapes: ((120,), (120,)), types: (tf.int64, tf.int64)>

In [29]:
for input_txt,target_txt in dataset.take(1):
  print(input_txt.numpy())
  print("".join(ind_to_char[input_txt.numpy()]))
  print('\n')
  print(target_txt.numpy())
  print("".join(ind_to_char[target_txt.numpy()]))


[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [30]:
batch_size =128

In [31]:
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [32]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

In [33]:
vocab_size = len(vocab)

In [34]:
vocab_size

84

In [35]:
embed_dim = 64

In [36]:
rnn_neurons = 1026

In [37]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [38]:

help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)
    Computes the sparse categorical crossentropy loss.
    
    Usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    Args:
      y_true: Ground truth values.
      y_pred: The predicted values.
      from_logits: Whether `y_pred` is expected to be a logits tensor. By default,
        we assume that `y_pred` encodes a probability distribution.
      axis: (Optional) Defaults to -1. The dimension along which the entropy is
        computed.
    
    Returns:
      Sparse categorical crossentropy loss value.



In [39]:
def sparse_cat_losses(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)#one hot encoded

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [41]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):

  model = Sequential()
  model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
  model.add(GRU(rnn_neurons,return_sequences=True,
                stateful=True,recurrent_initializer='glorot_uniform'))

  model.add(Dense(vocab_size))

  model.compile('adam',loss=sparse_cat_losses)
  return model

In [42]:
model = create_model(vocab_size=vocab_size,
                     embed_dim=embed_dim,
                     rnn_neurons=rnn_neurons,
                     batch_size=batch_size)

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [44]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)

In [45]:
example_batch_predictions.shape

TensorShape([128, 120, 84])

In [46]:
sample_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1 )

In [47]:
sample_indices = tf.squeeze(sample_indices,axis=-1).numpy()

In [48]:
sample_indices

array([19, 35,  1, 57, 62, 74,  2, 21, 52, 30, 36, 66, 63, 31, 58, 21, 29,
       19, 76, 57, 67,  7, 82, 58,  1, 70, 37, 64, 65, 50, 21, 27, 76, 21,
       83, 67, 38, 38, 17, 32, 48, 69, 71, 51, 29, 53, 75,  6, 18,  3, 62,
       39, 39, 70, 82, 49,  6, 24, 69, 32, 10, 64, 34, 66, 76, 50, 47, 71,
       70, 18, 48, 55, 80, 18, 38, 14, 35, 32,  8, 73, 69, 28, 42, 48, 73,
       68,  4, 24,  9, 64, 66, 71, 17,  9,  5, 13, 12, 81, 26, 29, 22, 35,
       67, 64, 26, 77, 54,  8, 77, 77, 76, 36, 29, 45, 81, 51, 29, 20, 18,
       52])

In [49]:
ind_to_char[sample_indices]

array(['8', 'J', ' ', 'b', 'g', 's', '!', ':', '[', 'E', 'K', 'k', 'h',
       'F', 'c', ':', 'D', '8', 'u', 'b', 'l', ')', '|', 'c', ' ', 'o',
       'L', 'i', 'j', 'Y', ':', 'B', 'u', ':', '}', 'l', 'M', 'M', '6',
       'G', 'W', 'n', 'p', 'Z', 'D', ']', 't', '(', '7', '"', 'g', 'N',
       'N', 'o', '|', 'X', '(', '>', 'n', 'G', '.', 'i', 'I', 'k', 'u',
       'Y', 'V', 'p', 'o', '7', 'W', '`', 'y', '7', 'M', '3', 'J', 'G',
       ',', 'r', 'n', 'C', 'Q', 'W', 'r', 'm', '&', '>', '-', 'i', 'k',
       'p', '6', '-', "'", '2', '1', 'z', 'A', 'D', ';', 'J', 'l', 'i',
       'A', 'v', '_', ',', 'v', 'v', 'u', 'K', 'D', 'T', 'z', 'Z', 'D',
       '9', '7', '['], dtype='<U1')

In [50]:
import os
os.chdir('/content/drive/My Drive/Trained models')

In [51]:
model = create_model(vocab_size=vocab_size,
                     embed_dim=embed_dim,
                     rnn_neurons=rnn_neurons,
                     batch_size=1)

In [52]:
from tensorflow.keras.models import load_model
#epochs = 50 
#model.fit(dataset,epochs=epochs)
model.load_weights('NLP_RNN_SHK.h5')

In [53]:
model.build(tf.TensorShape([1,None]))

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [60]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_index[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [62]:
print(generate_text(model,'JULIET',gen_size=1000))

JULIET for Master

Enter ANTIPHOLUS OF CASTARD and help smotterious Tometh.

  DULCH. This is Bolingbroke herself, hath sent to hernipht was lamp loud and honourable change,
    I humbly trade than thou tort. Thereby,
    Since I am sick and to be Rome,
   Nurse! O Assictorchies! He doth invite him more;
    For manifest perilous actions in my eyellsh.
  Dorg. What need you do? For such a
    subject to his hand, and to accuse it; an two gates my househamon sleep out.
  ROSALINE. With owes not yours.
  CASSIUS. I hear him curse.               Exit.



Scene III.
Lady. Yes, fool so fairly like you wed, this per piece of each one his honour;
    Since you redeem it in this court where you-
    And put the hell with Hector. Shall she now rises
    For thy redress, to change our strong I presently.
  TRANIO. How! The devil e'er
    Graces sword unto the chamber-door dield!
    I had no fat but it beg, and leap
    And not perform at that gird, if you be not.
  MARIA. That's to your people 